In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# %pip install --upgrade pip
# %pip install --disable-pip-version-check \
#     torch \
#     torchdata --quiet

# %pip install \
#     transformers \
#     datasets \
#     evaluate \
#     rouge_score \
#     loralib \
#     peft --quiet
!pip install phonemizer
!apt-get install espeak -y

!apt-get update
!apt-get install libespeak-dev
!pip install --upgrade torchaudio
!pip install git+https://github.com/speechbrain/speechbrain.git@main

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 21.8 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=b5f1870666c2fede221ff8319a54edf0f743cf6d2690263015ddf0c9a2449bfa
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
Reading package lists... Done
Building dependency tree       
Reading state information... 

In [4]:
from datasets import load_dataset, Audio
dataset = load_dataset("keithito/lj_speech")

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for keithito/lj_speech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/keithito/lj_speech
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/13100 [00:00<?, ? examples/s]

In [5]:
print(dataset.keys())

dict_keys(['train'])


In [6]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [7]:
train_dataset = dataset['train']
print(len(train_dataset))

13100


In [8]:
print(train_dataset[0])

{'id': 'LJ001-0001', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0001.wav', 'array': array([-0.00061751, -0.00074496, -0.00068972, ...,  0.00068615,
        0.00013802,  0.        ]), 'sampling_rate': 16000}, 'file': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0001.wav', 'text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition', 'normalized_text': 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition'}


In [9]:
import IPython.display as ipd
first_audio = train_dataset[0]['audio']
print(first_audio)

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/917ece08c95cf0c4115e45294e3cd0dee724a1165b7fc11798369308a465bd26/LJSpeech-1.1/wavs/LJ001-0001.wav', 'array': array([-0.00061751, -0.00074496, -0.00068972, ...,  0.00068615,
        0.00013802,  0.        ]), 'sampling_rate': 16000}


In [10]:
ipd.Audio(data=first_audio['array'], rate=first_audio['sampling_rate'])

In [11]:
# dataset_w = dataset.train_test_split(test_size=0.85)
dataset = train_dataset

In [12]:
num_samples = 1000
dataset = dataset.shuffle(seed=42).select(range(num_samples))

In [13]:
len(dataset)

1000

In [14]:
from transformers import AutoProcessor, AutoModelForTextToSpectrogram

processor = AutoProcessor.from_pretrained("microsoft/speecht5_tts")
model = AutoModelForTextToSpectrogram.from_pretrained("microsoft/speecht5_tts")

2024-06-11 09:59:19.372221: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 09:59:19.372406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 09:59:19.563883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

In [15]:
tokenizer = processor.tokenizer

In [16]:
# Function to extract all unique characters from the text
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset.column_names,
)

# Extract vocab from the dataset
dataset_vocab = set(vocabs["vocab"][0])

# Get vocab from the tokenizer
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

# Print the dataset and tokenizer vocabularies
print("Dataset Vocabulary:", dataset_vocab)
print("Tokenizer Vocabulary:", tokenizer_vocab)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset Vocabulary: {'W', 'T', 'M', 'R', 'n', '.', 't', ':', 'f', 'H', 'S', 'I', 'G', 'O', 'Z', 'K', 'x', 'e', ' ', 'Y', 'm', 'd', 'j', 'L', 'z', 'u', 'A', 'C', 'U', 'i', 'r', 'l', 'J', 'y', "'", 'D', 'c', '!', 'g', '?', ')', ';', 's', 'v', 'o', 'P', 'q', 'N', 'B', '"', 'V', 'F', 'w', 'k', '-', 'b', '(', 'a', 'h', 'p', ',', 'E', 'Q'}
Tokenizer Vocabulary: {'M', 't', 'I', 'm', 'L', 'A', 'C', 'X', 'i', 'J', '<unk>', '!', 'é', '"', 'V', '—', 'a', 'h', '}', '̄', 'Q', '<mask>', 'R', 'K', 'Y', '<ctc_blank>', 'j', 'z', 'l', 'y', "'", 'D', 'ê', ']', '<s>', ';', 'N', 'w', '-', '(', 'T', 'W', 'n', '.', ':', 'f', 'S', 'H', 'G', 'Z', 'x', 'd', '[', 'c', 'g', 's', 'v', 'o', 'B', 'œ', 'k', 'E', '▁', '{', 'æ', 'O', 'e', '/', 'u', 'U', 'r', '<pad>', '?', '</s>', ')', 'q', 'P', 'F', 'b', ',', 'p'}


In [17]:
dataset_vocab - tokenizer_vocab


{' '}

In [18]:
# Character replacements
replacements = [
    ("à", "a"),
    ("ç", "c"),
    ("è", "e"),
    ("ë", "e"),
    ("í", "i"),
    ("ï", "i"),
    ("ö", "o"),
    ("ü", "u"),
]

# Function to clean up text by replacing specified characters
def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
    return inputs

# Apply the cleanup_text function to the dataset
dataset = dataset.map(cleanup_text)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
# Print cleaned text
print(dataset[0]["normalized_text"])

and such projects as they approve will be next submitted to the President who under the Act is required to make final allocations.


In [20]:
# Function to print number of trainable model parameters
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 144431684
all model parameters: 144431684
percentage of trainable model parameters: 100.00%


In [21]:
from speechbrain.inference.speaker import EncoderClassifier

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [23]:
import torch
import os
device = "cuda" if torch.cuda.is_available() else "cpu"
speaker_model = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-xvect-voxceleb",
    run_opts={"device": device},
    savedir=os.path.join("/tmp", "speechbrain/spkrec-xvect-voxceleb"),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

hyperparams.yaml:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [24]:
MAX_LENGTH = 256

def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        text=example["normalized_text"],
        audio_target=audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector
    example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

    return example

In [25]:
import torch
processed_sample = prepare_dataset(dataset[0])
list(processed_sample.keys())

['input_ids', 'labels', 'speaker_embeddings']

In [26]:
processed_sample["speaker_embeddings"].shape

(512,)

In [27]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [28]:
dataset = dataset.train_test_split(test_size=0.1)

In [29]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [30]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [31]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=100,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)


In [ ]:
# start training process
trainer.train()